### Select microbes by correlation and pc1 score in digester 1, 3, 4

In [1]:
library(dplyr)
library(tibble)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
load('trimmed-150nts/correlation-analysis/processed_data.RData')
load('trimmed-150nts/correlation-analysis/cor_lag.RData')

In [3]:
chem_names <- c("pH","COD","PhosphorousR","PhosphorousT" ,"AmmoniaN", 
                "Alkalinity","NitrateN", "NitrogenT" ,"Volatile.Fatty.Acids" ,"TSS.VSS")
rownames(taxa) <- taxa$names
bac_cor_di134 <- list()
bac_pca_di134 <- list()
bac_selected <- list()
bac_info <- data.frame()

In [4]:
for(ch in 1:10){
  chem.name <- chem_names[ch]
  bac_cor <- list()
  for(i in c(1,3,4)){
    corr_tp <- corr[[i]]
    chem_tp <- chem_all[[i]]
    bac_tp <- bac_all[[i]]
    
    pos <- corr_tp %>% rownames_to_column('name') %>%
        top_n(100, wt=corr_tp[,chem.name]) %>%
        column_to_rownames('name')

    neg <- corr_tp %>% rownames_to_column('name') %>%
        top_n(-100, wt=corr_tp[,chem.name]) %>%
        column_to_rownames('name')
    
    name_pos <- rownames(pos)
    name_neg <- rownames(neg)
    
    bac_cor[[i]] <- c(name_pos, name_neg)
  }
  
  tbl <- table(unlist(bac_cor))
  names_table <- names(tbl)
  cor_sig <-  names_table[which(tbl==3)]
  bac_cor_di134[[ch]] <- cor_sig

  bac_pca <- list()
  pr_score <- list()
  for(i in c(1,3,4)){
    corr_tp <- corr[[i]]
    chem_tp <- chem_all[[i]]
    bac_tp <- bac_all[[i]]
    
    #pos cor group pca
    bac_pos <- bac_tp[, which(corr_tp[, chem.name] > 0)]
    bac_pos <- data.frame(t(bac_pos), check.names = F)
    pos.pr <- prcomp(bac_pos, scale. = T, center = T)
    pos.pr.x <- data.frame(pos.pr$x)
    
    pos_top <- pos.pr.x %>% rownames_to_column('name') %>% top_n(100, wt=abs(PC1)) %>%
                column_to_rownames('name')
    #neg cor group pca
    bac_neg <- bac_tp[, which(corr_tp[, chem.name] < 0)]
    bac_neg <- data.frame(t(bac_neg), check.names = F)
    neg.pr <- prcomp(bac_neg, scale. = T, center = T)
    neg.pr.x <- data.frame(neg.pr$x)
    
    neg_top <- neg.pr.x %>% rownames_to_column('name') %>% top_n(100, wt=abs(PC1)) %>%
                column_to_rownames('name')
    
    bac_pca[[i]] <- c(rownames(pos_top), rownames(neg_top))
    pr_score[[i]]<- rbind(pos.pr.x, neg.pr.x)
  } 
  tbl<-table(unlist(bac_pca))
  names_table <- names(tbl)
  pca_sig <-  names_table[which(tbl==3)]
  bac_pca_di134[[ch]] <- pca_sig
  
  namess <- intersect(cor_sig, pca_sig)
  if(length(namess)!=0) {
    bac_selected[[ch]] <- namess
    df <- taxa[which(taxa$names %in% namess),-c(1,2)]
    for(k in c(1,3,4)){
      score_temp <- pr_score[[k]][namess, ]$PC1
      corr_taxa_temp <- corr[[k]][namess, ][,chem.name]
      df_temp <- data.frame(score_temp, corr_taxa_temp, row.names = namess)
      colnames(df_temp) <- c(paste('Di',k,'pcscore',sep = '_'), 
                             paste('Di',k,'corr',sep = '_'))
      df <- cbind(df,df_temp)
    }
    bac_info <- rbind(bac_info, df)
  } else {
    bac_selected[[ch]] <- NULL
  }
}

In [5]:
dim(bac_info)
bac_info

[1] 26  9

,names,kingdom,phylum,Di_1_pcscore,Di_1_corr,Di_3_pcscore,Di_3_corr,Di_4_pcscore,Di_4_corr
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Bac20,Bac20,Bacteria,no_phylum_info,11.661703,0.7370316,11.438211,0.7518562,13.34626,0.6919752
Bac1604,Bac1604,Bacteria,Thermotogae,14.794651,0.6765463,14.336008,0.6979521,16.30557,0.6275678
Bac1817,Bac1817,Bacteria,Caldiserica,10.464908,0.7114169,11.249299,0.7481007,12.55034,0.6210237
Bac2720,Bac2720,Bacteria,no_phylum_info,12.205233,0.7978690,13.384621,0.7241072,14.87946,0.6930573
Bac3413,Bac3413,Bacteria,Thermotogae,12.052510,0.7631267,12.127472,0.7297400,13.97779,0.7193448
Bac4780,Bac4780,Bacteria,Thermotogae,11.840243,0.7659813,11.567041,0.8064563,13.63068,0.7049474
Bac5050,Bac5050,Bacteria,Caldiserica,15.076016,0.7250386,14.923893,0.6986029,16.86511,0.7206305
Bac5264,Bac5264,Bacteria,no_phylum_info,10.780241,0.7455778,11.599239,0.8078734,13.04983,0.6753663
Bac6132,Bac6132,Archaea,Euryarchaeota,8.586493,0.6677442,10.315056,0.7698908,11.58892,0.7733463


In [6]:
write.table(bac_info, file='trimmed-150nts/correlation-analysis/selected_microbes_info.txt', sep='\t')

In [7]:
table(unlist(bac_selected))
selected <- unlist(bac_selected)
selected <- selected[!duplicated(selected)]
selected


Bac1418 Bac1604 Bac1817   Bac20 Bac2720 Bac3003 Bac3413 Bac3778 Bac3779 Bac3859 
      1       2       1       1       1       1       1       1       2       1 
Bac4214 Bac4780 Bac5050 Bac5264 Bac5702 Bac6132 Bac6584 
      1       4       2       3       1       2       1 

[1] "Bac1604" "Bac1817" "Bac20"   "Bac2720" "Bac3413" "Bac4780" "Bac5050"
 [8] "Bac5264" "Bac6132" "Bac6584" "Bac3779" "Bac4214" "Bac5702" "Bac1418"
[15] "Bac3003" "Bac3778" "Bac3859"